In [2]:
!pip install -q langchain-openai langchain-community langchain-core requests duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests
import os

In [4]:
os.environ["OPENAI_API_KEY"] = "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"
openai_base_url = "https://openrouter.ai/api/v1"

In [5]:
llm = ChatOpenAI(
    openai_api_base=openai_base_url,
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model_name="deepseek/deepseek-chat-v3-0324:free",
    temperature=0.7
)

In [7]:
# till now we have imported the LLM and below we will make a simple AI Agent

In [12]:
# using this below tool and testing
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

result = search_tool.invoke("Top news in india today")

In [13]:
result

"India News | Latest India News | Read latest and breaking news from India. Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Amid rising tensions between Iran and Israel, an evacuation flight carrying Indian nationals is expected to land in India. Investigations continue into the Air India Flight AI 171 crash ... Zee News brings latest news from India and World on breaking news, today news headlines, politics, business, technology, bollywood, entertainment, sports and others. Find exclusive news stories on Indian politics, current affairs, cricket matches, festivals and events Read all India news and breaking news today from India on politics, business, entertainment, technology, sports, lifestyle and more at CNBC TV18. Join Us. ... Top India News. Mumbai's Most Wanted: Madhukar Zende's riveting memoir captures the city's darkest era and its brightest grit. Jun 15, ... Latest News: Stay updated with the latest news f

In [14]:
# test llm
llm.invoke("Hello")

AIMessage(content='Hello! 😊 How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 4, 'total_tokens': 16, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324:free', 'system_fingerprint': None, 'id': 'gen-1750264801-IrwofBSPFXgv7Dz2lwcZ', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--295ada69-9424-4e06-8ab6-bfbc9151238c-0', usage_metadata={'input_tokens': 4, 'output_tokens': 12, 'total_tokens': 16, 'input_token_details': {}, 'output_token_details': {}})

In [15]:
# now create the agent
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [16]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt
)

In [20]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    handle_parsing_errors=True
)

In [21]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "3 ways to reach goa from delhi"})
print(response)



> Entering new AgentExecutor chain...
To find the best ways to travel from Delhi to Goa, I should look for the most common transportation options available such as flights, trains, and buses. 

Action: duckduckgo_search  
Action Input: "3 best ways to travel from Delhi to Goa"

Conclusion: The best and cheapest way to travel from Delhi to Goa. As all ways to travel to Goa has its benefits. However, if you want to select the best way to travel it could be the Air and Train journey. Goa travelling by flight options are best if budget is not an issue. Flight (Air) is the most convenient way to reach Goa from Delhi. A tourist destination, Goa can be reached by air, rail or road from Delhi. Flight; The quickest way to get to Goa from Delhi is by flight. A flight to Goa takes around 2 hours to reach. Fares are generally in the range of Rs.2,000 to Rs.3,000. However, during the peak tourist season, fares can go up. Bus; There are no direct buses to Goa from ... Best Budget Strategy: If you 

In [24]:
response['output']

'**  \n\n1. **By Flight (Air Travel)**  \n   - **Fastest option** (2.5–3.5 hours).  \n   - **Cost:** ₹2,000–₹10,000 (book early for best prices).  \n\n2. **By Train**  \n   - **Scenic and budget-friendly** (24–39 hours).  \n   - **Cost:** ₹800–₹3,500 (Sleeper to AC classes).  \n   - **Best trains:** Rajdhani Express (25 hours), Goa Express.  \n\n3. **By Road (Bus or Self-Drive)**  \n   - **Flexible but longest** (~30+ hours).  \n   - **Bus options:** Private/state-run (cheaper but less comfy).  \n   - **Self-drive:** NH48 route (adventurous but tiring).  \n\n**Pro Tip:** Flights are best for speed, trains for budget + comfort, and road trips for flexibility. Book early to save money!  \n\nLet me know if you need help with specific bookings or routes.'

### **Now let's imporve the same code with some api and weather report**

In [31]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=86faa70fe3d75ff753&query={city}'  #go to website and find again

  response = requests.get(url)

  return response.json()

In [32]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [33]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [34]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [35]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
First, I need to find the capital of Madhya Pradesh. Then, I will use the capital's name to find its current weather condition.

Action: duckduckgo_search  
Action Input: "capital of Madhya Pradesh"  Indore (/ ɪ n ˈ d ɔːr / ⓘ; ISO: Indaura, Hindi: [ɪn̪d̪ɔːr]) is the largest and most populous city in the Indian state of Madhya Pradesh.The commercial capital of the state, it has been declared as the cleanest city of India 7 times in a row. [15] It is also considered the largest education hub in central India and houses campuses of both the Indian Institute of Technology and the ... Madhya Pradesh, state of India that is situated in the heart of the country. It has no coastline and no international frontier. Its physiography is characterized by low hills, extensive plateaus, and river valleys. The capital is Bhopal, in the west-central part of the state. Its capital is Bhopal, and Madhya Pradesh is the second-largest Indian state by area and the fif

In [36]:
response['output']

'The capital of Madhya Pradesh is Bhopal. The current weather in Bhopal is partly cloudy with a temperature of 27°C. The wind speed is 19 km/h coming from the SSW direction, humidity is at 100%, and the visibility is 6 km. It feels like 30°C outside. The air quality index is moderate.'